This notebook contains a few examples for bcc Fe to illustrate how VASP_job works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp() or self.run_job()

In [1]:
import os
from subprocess import run
import numpy as np
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

import nglview

In [2]:
def prepare_vasp(A, mu, repeat, name, value_of_m, kpoints, ntasks, time):

   # prepare folder
   cwd = dir_0 + "/" + name
   if not os.path.exists(cwd):
      run("mkdir " + cwd, shell=True)

   # prepare crystal structure
   a = 3.95 # A
   alpha = 90
   structure = crystal((A, 'Mn', 'N'),
                        basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                        spacegroup=221,
                        cellpar=[a, a, a, alpha, alpha, alpha])

   # prepare magnetism (triangular AFM state)
   Dtheta = 120 * np.pi/180
   theta = -Dtheta

   number_of_atoms = len(structure)
   magdirs   = []
   ms    = []
   # B_CONSTRs = []
   for i in range(number_of_atoms):
      if 'Mn' == structure.get_chemical_symbols()[i]:
         theta += Dtheta
         magdirs.append(np.array([np.cos(theta), np.sin(theta), 0])*mu)
         ms.append(value_of_m)
         # B_CONSTRs.append([0.0, 0.0, 0.0])
      else:
         magdirs.append([0.0, 0.0, 0.0])
         ms.append(1)
         # B_CONSTRs.append([0.0, 0.0, 0.0])

   structure.new_array("magdirs", magdirs, dtype=float)
   structure.new_array("ms", ms, dtype=float)
   # structure.new_array("B_CONSTR", B_CONSTR, dtype=float)

   # create vasp class
   vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path, pyscript=False)
   # update current working directory
   vasp.io.cwd = cwd

   # create SUPERCELL (should be done after adding arrays):
   structure = structure.repeat(repeat)

   # update kpoints
   vasp.structure.kpoints = kpoints

   # set calculation, which creates all input files
   vasp.set_calculation(structure, ntasks=ntasks, time=time)

   return vasp

In [3]:
dir_0 = os.getcwd()

In [4]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# supercell

## 2x2x2

In [5]:
A = "Ga"
mu = 3

repeat = (2,2,2)
name = "2x2x2"
value_of_m = 1
kpoints = "1 1 1"
ntasks = 2*40
time = 1*24*60
vasp = prepare_vasp(A, mu, repeat, name, value_of_m, kpoints, ntasks, time)

In [6]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Ga,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
5,Ga,"(0.0, 0.0, 3.95)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
10,Ga,"(0.0, 3.95, 0.0)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
15,Ga,"(0.0, 3.95, 3.95)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
20,Ga,"(3.95, 0.0, 0.0)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
25,Ga,"(3.95, 0.0, 3.95)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
30,Ga,"(3.95, 3.95, 0.0)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
35,Ga,"(3.95, 3.95, 3.95)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
3,Mn,"(1.975, 1.975, 0.0)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"
8,Mn,"(1.975, 1.975, 3.95)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"


## 4x4x4

In [7]:
A = "Ni"
mu = 3

repeat = (4,4,4)
name = "4x4x4"
value_of_m = 1
kpoints = "1 1 1"
ntasks = 5*40
time = 4*24*60
vasp = prepare_vasp(A, mu, repeat, name, value_of_m, kpoints, ntasks, time)

In [8]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
3,Mn,"(1.975, 1.975, 0.0)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"
8,Mn,"(1.975, 1.975, 3.95)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"
13,Mn,"(1.975, 1.975, 7.9)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"
18,Mn,"(1.975, 1.975, 11.850000000000001)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"
23,Mn,"(1.975, 5.925000000000001, 0.0)","(-1.5000000000000013, -2.598076211353315, 0.0)",1.0,inf,"(-1.5000000000000013, -2.598076211353315, 0.0)","(0.0, 0.0, 0.0)"
...,...,...,...,...,...,...,...
295,Ni,"(11.850000000000001, 7.9, 11.850000000000001)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
300,Ni,"(11.850000000000001, 11.850000000000001, 0.0)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
305,Ni,"(11.850000000000001, 11.850000000000001, 3.95)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
310,Ni,"(11.850000000000001, 11.850000000000001, 7.9)","(0.0, 0.0, 0.0)",1.0,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"


In [9]:
vasp.df["positions"].values

array([(1.975, 1.975, 0.0), (1.975, 1.975, 3.95), (1.975, 1.975, 7.9),
       (1.975, 1.975, 11.850000000000001),
       (1.975, 5.925000000000001, 0.0), (1.975, 5.925000000000001, 3.95),
       (1.975, 5.925000000000001, 7.9),
       (1.975, 5.925000000000001, 11.850000000000001),
       (1.975, 9.875, 0.0), (1.975, 9.875, 3.95), (1.975, 9.875, 7.9),
       (1.975, 9.875, 11.850000000000001),
       (1.975, 13.825000000000001, 0.0),
       (1.975, 13.825000000000001, 3.95),
       (1.975, 13.825000000000001, 7.9),
       (1.975, 13.825000000000001, 11.850000000000001),
       (5.925000000000001, 1.975, 0.0), (5.925000000000001, 1.975, 3.95),
       (5.925000000000001, 1.975, 7.9),
       (5.925000000000001, 1.975, 11.850000000000001),
       (5.925000000000001, 5.925000000000001, 0.0),
       (5.925000000000001, 5.925000000000001, 3.95),
       (5.925000000000001, 5.925000000000001, 7.9),
       (5.925000000000001, 5.925000000000001, 11.850000000000001),
       (5.925000000000001, 9.8

visualize structure

In [10]:
view = nglview.show_ase(vasp.io.structure_ase)
view.add_unitcell()
view

NGLWidget()